# Exercise 2: Merging and Deriving New Columns

In [1]:
### Now open data in this notebook: An Intro to F Strings
* We can read in the Excel workbook above using the URL:
`df = pd.read_excel("gs://calitp-analytics-data/data-analyses/starter_kit/starter_kit_csis_scoring_workbook.xlsx")`
* However, this is a bit of an eyesore. Additionally, you might want to save your finished work  into the folder `starter_kit`. 
* This is where f-strings come in. Wht are f-strings? 
> Python f-strings provide a quick way to interpolate and format strings. They’re readable, concise, and less prone to error than traditional string interpolation and formatting tools...
    * Read more about them [here](https://realpython.com/python-f-strings/#f-strings-a-new-and-improved-way-to-format-strings-in-python).

* Here's an example:
`base_url = "https://www.instagram.com/"` 
`first_profile = "zendaya"`
    * When you combine them, as `f"{base_url}{first_profile}"` it will yield `https://www.instagram.com/zendaya`.
    * What if you want another instagram profile? Simply combine your second_profile with the base_url.  
    `second_profile = anyataylorjoy`
    `f"{base_url}{second_profile}"`
    
    

SyntaxError: invalid syntax (3914705476.py, line 2)

In [ ]:
## Merging 
* Your manager asks you to present the average overall score, the max score, and the min score by each Caltrans District.
* Annoyingly enough, the `overall_score` column and the `ct_district` are in two different dataframes.
* Welcome to DDS! This will happen to you all the time starting now. 
* Thankfully, merging couldn't be simpler...some of the time...

In [ ]:
## Make a Chart 
* Read in the parquet file from GCS using F Strings.
* Make a visualization using one `Altair` (link to Altair)
    * Requirements: use color blind friendly palette (link to doc)
    * The chart must be in Century-Gothic font (link to CSIS example)
    * Size it to be 600 pixels wide, 400 pixels high